<img heigth="8" src="https://i.imgur.com/826AqJI.png" alt="pbs-enae">

<h1 align="left">Análisis de datos con Python</h1>

<h2 align="left"><i>Estrategias de marketing basadas en datos</i></h2>

<p align="left">
  <h3>Joseph F. Vergel-Becerra | Machine Learning - Tools and Skill Courses</h3>
  <br>
  <b>Last updated:</b> <i>16/02/2023</i>
  <br><br>
  <!-- <a href="#tabla-de-contenido">Tabla de contenido</a> • -->
  <a href="#referencias">Referencias</a> •
  <a href="#contribuir">Contribuir</a>
  <!-- <a href="#agradecimientos">Agradecimientos</a> -->
  <br><br>
</p>
<table align="left">
  <td>
      <a href="https://img.shields.io/badge/version-0.1.0-blue.svg?cacheSeconds=2592000">
        <img src="https://img.shields.io/badge/version-0.1.0-blue.svg?cacheSeconds=2592000" alt="Version" height="18">
      </a>
  </td>
  <td>
    <a href="https://colab.research.google.com/github/joefavergel/pbs-enae-python-beginners-course/blob/main/4-python-data-analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
      </a>
  </td>
  <td>
    <a href="https://github.com/joefavergel/pbs-enae-ml-course" target="_parent"><img src="https://img.shields.io/github/forks/joefavergel/pbs-enae-ml-course?style=social" alt="Fork"/>
      </a>
  </td>
</table>
<br>
<br>

---

Ofrecer un servicio o producto a los clientes con un perfil de posible comprador, constituye un factor diferenciador en las estrategias comerciales y de marketing de las compañías e impacta drásticamente en el ahorro de recursos de estas divisiones. Es así como se define el concepto de *target* u objetivo en marketing, que no es más que la definición de los clientes potenciales o público general, que tiene como objetivo una determinada estrategia de marketing, a la hora de hacerles llegar los  productos o servicios de la compañía. En esta oportunidad **representaremos el brazo tecnológico de la división de marketing y comercialización de una institución bancaria** y plantearemos una solución predictiva sobre datos de campañas de marketing telefónico, con la que **predeciremos la probabilidad de que un cliente contrate un certificado de depósito a plazo fijo (CDP)**. De esta manera podremos ofrecerle a nuestros clientes con mayor probabilidad de suscripción, **campañas publicitarias acertadas que maximizen la captación de nuevos contratos** y que al mismo tiempo **reduzcan los costos publicitarios**.

<a id='tabla-de-contenido'></a>
## Tabla de contenido
    
<ol>
    <li><a href="#1-preludio"><b>Preludio</b></a></li>
    <li><a href="#2-ingesta"><b>Ingesta de datos</b></a></li>
    <li><a href="#3-analisis"><b>Análisis exploratorio y entendimiento de los datos</b></a></li>
    <li><a href="#4-descriptiva">Estadística descriptiva</a></li>
    <ol type="a">
        <li><a href="#i-tendencia">Medidas de tendencia central</a></li>
        <li><a href="#ii-dispersion">Medidas de dispersión</a></li>
        <li><a href="#iii-correlacion">Correlación y covarianza</a></li>
    </ol>
    <li><a href="#5-calidad">Análisis de calidad de los datos</a></li>
    <ol type="a">
        <li><a href="#i-faltantes">Detección de valores faltantes</a></li>
        <li><a href="#ii-outliers">Detección de outliers</a></li>
    </ol>
    <li><a href="#ejercicios"><b>Ejercicios caso práctico: "Análisis de Datos con Python"</b></a></li>
</ol>

<a id="1-preludio"></a>
## 1. Preludio

In [ ]:
import sys
from IPython.core.display import HTML

from pathlib import Path
from packaging import version
import sklearn
import urllib.request


print("[INFO] Este proyecto requiere python 3.8 o superior y Scikit-Learn 1.0.1 o superior.")
assert sys.version_info >= (3, 8)
assert version.parse(sklearn.__version__) >= version.parse("1.0.1")
print("[INFO] Versiones vificadas exitosamente!")


def css_styling():
    styles_path = Path(f"./styles/custom.css")
    if not styles_path.is_file():
        Path("styles").mkdir(parents=True, exist_ok=True)
        url = f"https://github.com/joefavergel/pbs-enae-ml-course/blob/main/styles/custom.css?raw=true"
        urllib.request.urlretrieve(url, styles_path)

    styles = open("./styles/custom.css", "r").read()
    return HTML(styles)


css_styling()

<a id="2-ingesta"></a>
## 2. Ingesta de datos

La ingesta de datos a través de archivos es una práctica común en la gestión de información. Consiste en la importación de datos desde archivos, ya sean de texto, hojas de cálculo, bases de datos o cualquier otro formato, a un sistema o aplicación para su posterior procesamiento y análisis. Esta técnica es esencial para la mayoría de las empresas y organizaciones, ya que les permite recolectar y gestionar grandes cantidades de información de manera eficiente. En esta ocasión ingestaremos un base de datos de clientes del sector bancario.

In [ ]:
from pathlib import Path
from zipfile import ZipFile
import urllib.request

DATASET = "banking-marketing-targets"
DATA_PATH = f"datasets/{DATASET}/"


def download_dataset(dataset: str):
    zipfile_path = Path(f"datasets/{dataset}.zip")
    if not zipfile_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = f"https://github.com/joefavergel/datasets/blob/main/{dataset}.zip?raw=true"
        urllib.request.urlretrieve(url, zipfile_path)
    Path(f"datasets/{dataset}").mkdir(parents=True, exist_ok=True)
    try:
        ZipFile(zipfile_path).extractall(f"datasets/{dataset}")
        print(f"[INFO] Dataset \'{dataset}\' downloaded and uncompressed correctly!")
    except Exception as e:
        print(f"[Exception] There's been a problem: {e}")


download_dataset(dataset=DATASET)

In [ ]:
import os

print(
    f"[INFO] Los archivos presentes en \"{DATA_PATH}\":"
    f"\n\n{os.listdir(DATA_PATH)}"
)

In [ ]:
import os

import pandas as pd


train = pd.read_csv(os.path.join(DATA_PATH, "train.csv"), sep=";")
test = pd.read_csv(os.path.join(DATA_PATH, "test.csv"), sep=";")

train.rename(columns={"y": "cd"}, inplace=True)
test.rename(columns={"y": "cd"}, inplace=True)

with pd.option_context('display.max_columns', None):
    print(f"[INFO] Training dataset dimnesions (rows, cols): {train.shape}")
    display(train.head())
    
    print(f"\n[INFO] testing dataset dimnesions (rows, cols): {test.shape}")
    display(test.head())

<a id="3-analisis"></a>
## 3. Análisis exploratorio y entendimiento de los datos

El analisis exploratorio de los datos (EDA por sus siglas en ingles) es como comunmente se denomina de forma tecnica a la fase de "Descubra y visualice los datos para obtener información".

### Información de las variables del *dataset*

Datos del cliente:

1. `edad` (numérico): edad del cliente.
2. `job` (categórico): tipo de trabajo.
3. `marital` (categórico): estado civil.
4. `educación` (categórico): Nivel de escolaridad (studios primarios, secundarios o terciarios).
5. `default` (*booleano*): ¿tiene algun credito en mora?
6. `balance` (numérico): saldo promedio anual, en euros.
7. `housing` (*booleano*): ¿tiene préstamo de vivienda?.
8. `loan` (*booleano*): ¿tiene préstamo personal?.

Las siguientes variables estan relacionadas con el último contacto de la campaña de marketing actual:

9. `contact` (categórico): tipo de comunicación con el cliente.
10. `day` (numérico): último día de contacto del mes.
11. `month` (categórico): último mes de contacto del año.
12. `duration` (numérico): duración del último contacto, en segundos.

Otros atributos de las instancias:

13. `campaign` (numérico): número de contactos realizados durante esta campaña y para este cliente, incluye último contacto.
14. `pdays` (numérico): número de días que transcurrieron desde la última vez que se contactó al cliente en una campaña anterior, donde -1 significa que el cliente no fue contactado previamente.
15. `previous` (numérico): número de contactos realizados antes de esta campaña y para este cliente 
16. `poutcome` (categórico): resultado de la campaña de marketing anterior.

Variable objetivo (*target*) a predecir:

17. `cd` (*booleano*): ¿El cliente ha suscrito un depósito a plazo?.

In [ ]:
print(train.info())

In [ ]:
print(test.info())

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
print("[INFO] Columns of trainig dataset: \n", train.columns)
print("\n[INFO] Columns of trainig dataset: \n", test.columns)
print(
    "\n[INFO] Difference between training columns set and testing columns set: \n",
    set(train.columns) - set(test.columns)
)

<a id="4-descriptivaa"></a>
### 4. Estadistica descriptiva

La estadística descriptiva es importante en el análisis de datos porque ***proporciona una manera sistemática de resumir y visualizar grandes conjuntos de datos***. Permite identificar patrones, tendencias y características de los datos, como ***la media, la mediana y la desviación estándar***. Esto puede ayudar a entender mejor los datos y a tomar decisiones informadas basadas en la información que proporcionan. Además, ***la estadística descriptiva es una herramienta esencial para comunicar los hallazgos*** de una manera clara y efectiva a otras personas.

**Visualización de conteos de datos e histogramas de variables numéricas continuas**

La importancia de la visualización de variables categóricas y de variables numéricas continuas se puede resumir en los siguientes puntos:

- Ayuda a comprender y comunicar patrones y tendencias en los datos.
- Las variables categóricas representan características o atributos que se dividen en categorías o grupos discretos, mientras que las variables numéricas continuas representan valores que se pueden medir y se ubican en una escala continua.
- Al visualizar estas variables, se pueden identificar patrones, tendencias y relaciones en los datos.
- Esto permite tomar decisiones más informadas y precisas.
- Las visualizaciones pueden ayudar a comunicar los hallazgos a otras personas de manera clara y efectiva.

In [ ]:
!pip install seaborn statsmodels

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

high_carinality = ["duration", "pdays", "age", "balance"]
categorical = train.select_dtypes(include=['object', 'int']).columns.values
categorical = list(set(categorical) - set(high_carinality))
features = categorical + high_carinality

for column in features:
    plt.figure(figsize=(8, 4))
    if column in categorical:
        count_plot = sns.countplot(x=train[column], data=train)
        for p in count_plot.patches:
            count_plot.annotate(
                format(p.get_height()),
                (p.get_x() + p.get_width() / 2., p.get_height()),
                ha = 'center',
                va = 'center',
                xytext = (0, 10),
                textcoords = 'offset points'
            )
        plt.title(f"\"{column}\" Counts")
        plt.xticks(rotation=45)
        plt.show()
    else:
        dist_plot = sns.displot(data=train, x=column)
        plt.title(f"\"{column}\" Distribution")
        plt.xticks(rotation=45)
        plt.show()


Como la mayoría de los datos presentes en el *dataset* corresponden a variables categóricas, es necesario transformar primero dichas variables a numéricas y posterior a esto, volver a efectuar EDA sobre los datos transformados.

<a href="i-tendencia"></a>
### 4.a. Medidas de tendencia central

<p align="center">
  <a target="_blank">
    <img width="700px" src="https://i.imgur.com/KeJ6lbW.png" alt="standard-correlation">
  </a>
</p><br><br>

In [ ]:
train.mean(numeric_only=True)

In [ ]:
pd.DataFrame(train.median(numeric_only=True)).rename(columns={0: "median"})

In [ ]:
train.mode(numeric_only=True)

<a href="#ii-dispersion"></a>
### 4.b. Medidas de dispersión

<p align="center">
  <a target="_blank">
    <img width="700px" src="https://i.imgur.com/td7Vr3c.png" alt="standard-correlation">
  </a>
</p><br><br>

Una desviación estándar baja indica que la mayor parte de los datos de una muestra tienden a estar agrupados cerca de su media (también denominada el valor esperado), mientras que una desviación estándar alta indica que los datos se extienden sobre un rango de valores más amplio.

In [ ]:
train.std(numeric_only=True)

La desviación estándar es la raíz cuadrada positiva de la varianza:

In [ ]:
train.var(numeric_only=True)

Hay que tener en cuenta que la varianza puede verse muy influida por los valores atípicos y no se aconseja su uso cuando las distribuciones de las variables aleatorias tienen colas pesadas.

<a href="iii-correlacion"></a>
### 4.c. Correlación y covarianza

El análisis de correlaciones es importante en el análisis de datos porque permite identificar ***si existe una relación estadística significativa entre dos o más variables***. La ***la correlación puede ser positiva, negativa o nula***, y puede variar en fuerza y dirección. Al identificar la correlación entre variables, es posible comprender mejor cómo se relacionan y cómo afectan mutuamente.

***Las correlaciones puede proporcionar información valiosa para la toma de decisiones y la planificación de estrategias***. Por ejemplo, puede ayudar a identificar variables que tienen un impacto significativo en el rendimiento de una empresa o a predecir cómo cambiará una variable si cambia otra.

Ahora, sabiendo la importancia de las correlaciones revisemos el coficiente de correlacion lineal de Pearson:

<p align="center">
  <a target="_blank">
    <img width="700px" src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d4/Correlation_examples2.svg/1280px-Correlation_examples2.svg.png" alt="standard-correlation">
  </a><br><br>
  <b>Figura 1:</b> <i>Varios conjuntos de puntos (x, y), con el coeficiente de correlación de x e y para cada conjunto. La correlación refleja la fuerza y la dirección de una relación lineal (fila superior), pero no la pendiente de esa relación (centro), ni muchos aspectos de las relaciones no lineales (parte inferior). N.B.: la figura del centro tiene pendiente 0 pero en ese caso el coeficiente de correlación no está definido porque la varianza de Y es cero</i>¹⁶.
</p><br>

El coeficiente de correlación de Pearson va de –1 a 1. Cuando está cerca de 1, significa que hay una fuerte correlación positiva. Cuando el coeficiente es cercano a –1, significa que existe una fuerte correlación negativa o correlacion inversa. Finalmente, los coeficientes cercanos a 0 significan que no existe una correlación lineal. La Fig. 1 muestra varias distirbuciones de datos junto con el coeficiente de correlación entre sus ejes horizontal y vertical.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


def plot_corr(data: pd.DataFrame, title: str, method: str = "pearson", annot: bool = True):
    f,ax = plt.subplots(figsize=(10,8))
    sns.heatmap(
        data.corr(method=method),
        vmin=-1,
        vmax=1,
        annot=annot,
        linewidths=.5,
        fmt='.2f',
        ax=ax
    )
    plt.title(f"{method.title()}'s Correlation Coefficient between {title}")
    return ax

In [ ]:
ax = plot_corr(
    train.select_dtypes("number"),
    title="the Main Numeric Features",
    method="pearson",
    annot=True
)
plt.show()

Observamos que las correlaciones mas fuertes existe entre las variables "pdays" y "previous" y las variables "day" y "campaign":

In [ ]:
import numpy as np
import scipy


def plot_scatter_and_corr(
    dataframe: pd.DataFrame,
    x_feature: str,
    y_feature:str,
    ax: np.array = None,
    pos: int = None
):
    if ax is None:
        f, ax = plt.subplots(1, 2, figsize=(10,4))

    pos = 0 if pos is None else 1

    r, _ = scipy.stats.pearsonr(
        dataframe[x_feature].tolist(), dataframe[y_feature].tolist()
    )
    tau, _ = scipy.stats.kendalltau(
        dataframe[x_feature].tolist(), dataframe[y_feature].tolist()
    )
    ax[pos].scatter(
        dataframe[x_feature].tolist(),
        dataframe[y_feature].tolist(),
        alpha=0.2,
        color="#FC5185" if pos == 1 else "#364F6B",
        label=f"r={np.round(r, 2)}, tau={np.round(tau, 2)}"
    )
    ax[pos].set_xlabel(x_feature)
    ax[pos].set_ylabel(y_feature)
    ax[pos].set_title(f"{y_feature} vs. {x_feature}")
    ax[pos].tick_params(axis='x', rotation=45)
    ax[pos].legend()

    plt.tight_layout()

    return ax

In [ ]:
x_feature, y_feature = "pdays", "previous"
ax = plot_scatter_and_corr(
    dataframe=train, x_feature=x_feature, y_feature=y_feature,
)

x_feature, y_feature = "day", "campaign"
ax = plot_scatter_and_corr(
    dataframe=train, x_feature=x_feature, y_feature=y_feature, ax=ax, pos=1
)

plt.show()

In [ ]:
def corr_upper_triangular_dot(*args, **kwargs):
    corr_tau = args[0].corr(args[1], method=method)
    corr_text = f"{corr_tau:2.2f}".replace("0.", ".")
    ax = plt.gca()
    ax.set_axis_off()
    marker_size = abs(corr_tau) * 10000
    ax.scatter(
        [.5],
        [.5],
        marker_size,
        [corr_tau],
        alpha=0.6,
        cmap="coolwarm",
        vmin=-1,
        vmax=1,
        transform=ax.transAxes
    )
    font_size = abs(corr_tau) * 40 + 5
    ax.annotate(
        corr_text, [.5, .5,], 
        xycoords="axes fraction",
        ha='center',
        va='center',
        fontsize=font_size
    )
    return ax

In [ ]:
# np.seterr(invalid='ignore', over='ignore')

method = "pearson"
g = sns.PairGrid(train, aspect=1.0, diag_sharey=False)
g.map_lower(sns.regplot, lowess=True, ci=None, line_kws={'color': 'black'})
g.map_diag(sns.histplot, kde=False, bins=10)
g.map_upper(corr_upper_triangular_dot, method=method)
g.fig.suptitle(
    f"{method.title()}'s Correlation Coefficient between Dataset Features",
    fontsize=32
)
plt.show()

<a id="5-calidad"></a>
# 5. Análisis de la calidad de Datos

Análisis de la Calidad de los Datos en el Contexto de un Curso de Python y Análisis de Datos:

El "Análisis de la Calidad de los Datos" se refiere al proceso de inspeccionar y asegurar que los datos empleados para análisis sean de alta calidad, lo que significa que sean precisos, confiables y adecuados para el propósito del análisis. Dentro de este contexto, se pueden destacar dos pasos cruciales:

a. Detección de datos faltantes
b. Detección de *outliers* (valores atípicos)

<a id="i-faltantes"></a>
## 5.a. Detección de valores faltantes

 El análisis de valores faltantes es importante en el análisis de datos porque los valores faltantes pueden ***afectar la validez y la precisión de los resultados del análisis***. Los valores faltantes pueden ocurrir por diversas razones, como errores en la entrada de datos o la falta de respuesta a una pregunta en una encuesta. Al analizar los valores faltantes, es posible identificar patrones o sesgos en los datos y tomar medidas para corregirlos. En general, el análisis de valores faltantes es esencial para ***asegurar la integridad y confiabilidad de los resultados del análisis de datos***.

In [ ]:
!pip install missingno prettytable

In [ ]:
import numpy as np


train.replace("unknown", np.nan, inplace=True)
test.replace("unknown", np.nan, inplace=True)

In [ ]:
import missingno as msno
from prettytable import PrettyTable


def quantify_missing(dataframe: pd.DataFrame, tol: float):
    missing_value = dataframe.isnull().mean() * 100
    missing_value_dict = missing_value.to_dict()
    missing_percentage = missing_value > tol
    missing_percentage = missing_percentage.to_dict()
    count = 0
    table = PrettyTable(
        ['Feature', 'Missing Value Percentage'], float_format='3.2'
    )
    features = []
    for key in missing_percentage:
        if missing_percentage[key]:
            table.add_row([key, np.round(missing_value_dict[key], 8)])
            count = count + 1
            features.append(key)
    print(table)
    print('Total quantified features:', count)

In [ ]:
print(f"[INFO] Percentages of missing values for the training dataset (rows: {train.shape[0]}):")
quantify_missing(dataframe=train, tol=0.0)

print(f"\n[INFO] Percentages of missing values for the testing dataset (rows: {test.shape[0]}):")
quantify_missing(dataframe=test, tol=0.0)

In [ ]:
print(f"[INFO] Analysis of missing values for the training dataset (rows: {train.shape[0]}):")
msno.bar(train)
plt.show()

msno.matrix(train)
plt.show()

msno.heatmap(train)
plt.show()

In [ ]:
train["contact"].value_counts(normalize=True, dropna=False)

In [ ]:
train["poutcome"].value_counts(normalize=True, dropna=False)

<a id="ii-outliers"></a>
## 5.b. Detección de *outliers*

**Detección de Anomalías Univariada: Método de la desviación estándar**

<p align="center">
  <a target="_blank">
    <img width="400px" src="https://upload.wikimedia.org/wikipedia/commons/thumb/2/22/Empirical_rule_histogram.svg/1280px-Empirical_rule_histogram.svg.png" alt="a-b">
  </a><br>
  <b>Figura 6:</b> <i>Para una distribución aproximadamente normal, los valores comprendidos en un intervalo de semiancho una desviación típica respecto a la media aritmética de la muestra, abarcan el 68% de los datos; mientras que dos desviaciones de semiancho abarcan el 95%; y tres deviaciones de semiancho incluyen el 99.7%. Los porcentajes mostrados son probabilidades teóricas redondeadas, destinadas solo a aproximar los datos empíricos derivados de una población normal.</i>²²
</p><br><br>

---

²² <a href="https://es.wikipedia.org/wiki/Regla_68-95-99.7">Regla 68-95-99.7</a>.

In [ ]:
from copy import deepcopy

N_SAMPLES = len(train)

subtrain = deepcopy(train.sample(N_SAMPLES))
subtrain.reset_index(drop=True, inplace=True)

In [ ]:
target = "balance"
data = subtrain[target].tolist()
sigma = 5
data_mean, data_std = np.mean(data), np.std(data)
cut_off = data_std * sigma
lower, upper = data_mean - cut_off, data_mean + cut_off
outlier_flag = pd.Series([True if (x < lower or x > upper) else False for x in data ])
print(outlier_flag.value_counts(normalize=True))
outliers = [x for x in data if (x < lower or x > upper)]
print(
    "\n[INFO] Some possible outliers according to the standard deviation method for the "
    f"target \"{target}\"\n", np.random.choice(outliers, 20)
)

In [ ]:
print(f"[INFO] Skewness of \"{target}\": %f" % subtrain[target].skew())
print(f"[INFO] Kurtosis of \"{target}\": %f" % subtrain[target].kurt())

Los datos se consideran Gaussianos si la asimetría (*Skewness*) está entre -2 y +2 y la curtosis está entre -7 y +7.²²<br><br>

---

²² <a href="https://imaging.mrc-cbu.cam.ac.uk/statswiki/FAQ/Simon#:~:text=The%20values%20for%20asymmetry%20and,between%20%E2%80%907%20to%20%2B7.">Testing normality including skewness and kurtosis</a>.

**Detección de Anomalías Univariada:** ***Machine Learning***

In [ ]:
from sklearn.ensemble import IsolationForest

isolation_forest = IsolationForest(n_estimators=100, random_state=42)
isolation_forest.fit(subtrain[target].values.reshape(-1, 1))
xx = np.linspace(subtrain[target].min(), subtrain[target].max(), len(subtrain)).reshape(-1,1)
anomaly_score = isolation_forest.decision_function(xx)
outlier_flag = isolation_forest.predict(xx)

plt.figure(figsize=(10,4))
plt.plot(xx, anomaly_score, label='Anomaly score')
plt.fill_between(
    xx.T[0],
    np.min(anomaly_score),
    np.max(anomaly_score), 
    where=outlier_flag==-1,
    color='r', 
    alpha=.4,
    label='Outlier region'
)
plt.legend()
plt.ylabel('Anomaly score')
plt.xlabel(target)
plt.show()

In [ ]:
outlier_flag = pd.Series(outlier_flag).replace([-1, 1], [False, True])
print(outlier_flag.value_counts(normalize=True))
print(
    "\n[INFO] Some possible outliers according to the standard deviation methodod for the "
    f"target \"{target}\"\n", subtrain[outlier_flag][target].sample(20).tolist()
)

<a id="ejercicios"></a>
## 6. Ejercicios caso práctico: "Análisis de Datos con Python"

1. Replique la sección "Análisis Exploratorio de los datos" presentada en este `Jupyter notebook`, para el *dataset* que se descargará y descomprimirá  al ejecutar la siguiente celda de codigo:

In [ ]:
from pathlib import Path
from zipfile import ZipFile
import urllib.request

DATASET = "titanic-dataset"
DATA_PATH = f"datasets/{DATASET}/"


def load_traffic_congestion_data(dataset: str):
    zipfile_path = Path(f"datasets/{dataset}.zip")
    if not zipfile_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = f"https://github.com/joefavergel/datasets/blob/main/{dataset}.zip?raw=true"
        urllib.request.urlretrieve(url, zipfile_path)
    Path(f"datasets/{dataset}").mkdir(parents=True, exist_ok=True)
    try:
        ZipFile(zipfile_path).extractall(f"datasets/{dataset}")
        print(f"[INFO] Dataset \'{dataset}\' downloaded and uncompressed correctly!")
    except Exception as e:
        print(f"[Exception] There's been a problem: {e}")


load_traffic_congestion_data(dataset=DATASET)

pd.read_csv("datasets/titanic-dataset/titanic.csv")

<a id='referencias'></a>
## Referencias

[1] Halswanter, T. (2016). *An Introduction to Statistics with Python: With Applications in the Life Sciences.* Springer.

[2] Géron, A. (2022). *Hands-on machine learning with Scikit-Learn, Keras, and TensorFlow.* " O'Reilly Media, Inc.".

[3] VanderPlas, J. (2016). *Python data science handbook: Essential tools for working with data.* " O'Reilly Media, Inc.".

[4] Aggarwal, C. C. (2015). *Data mining: the textbook (Vol. 1)*. New York: springer.

<a id='contribuir'></a>
## Contribuir

<p>Para correcciones, <i>bugs</i> o sugerencias, por favor escribe a <a href="mailto:joefavergel@gmail.com">joefavergel@gmail.com</a> o directamente en el repositorio.</p>